## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## План исследования: <a id="step0"></a>

[**Шаг 1. Загрузка файла с данными, изучение общей информации**](#step1)

[**Шаг 2. Предобработка данных**](#step2)

[**Шаг 3. Ответы на вопросы исследования**](#step3)

- 3.1 Есть ли зависимость между наличием детей и возвратом кредита в срок?    
- 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?   
- 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?    
- 3.4 Как разные цели кредита влияют на его возврат в срок?
    
[**Шаг 4. Общий вывод и рекомендации банку**](#step4)
    

### Шаг 1. Открываем файл с данными и изучаем общую информацию. <a id="step1"></a>

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

df = pd.read_csv('/Users/maxim/Documents/Edu/DA/Yandex/Datasets/02_borrowers_reliability/data.csv')
df.head(10)

Installing mystem to /Users/maxim/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


Получаем общую информацию о датафрейме:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

На основании предварительного изучения данных необходимо устранить следующие проблемы, препятствующие анализу:
- неинформативные названия некоторых столбцов;
- отрицательные значения;
- отличающийся регистр записи однотипных данных;
- отсутствие строк и "ненаглядный" тип данных для столбцов трудового стажа и ежемесячного дохода.

### Шаг 2. Произведем предобработку данных  <a id="step2"></a>
[*Вернуться к плану*](#step0)

### Переименование столбцов

In [4]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [5]:
columns_new_names = ['children_number', 'days_employed', 'age_in_years', 'education_level', 'education_id', 'family_status', 'family_status_id', 'gender', 'employment_type', 'loan_debt_history', 'monthly_income', 'loan_purpose']
df.set_axis(columns_new_names, axis='columns', inplace=True)

In [6]:
df.columns

Index(['children_number', 'days_employed', 'age_in_years', 'education_level',
       'education_id', 'family_status', 'family_status_id', 'gender',
       'employment_type', 'loan_debt_history', 'monthly_income',
       'loan_purpose'],
      dtype='object')

### Вывод
Столбцы переименованы, смысл заголовков понятен, можно двигаться дальше.

### Обработка пропусков

Находим пропуски, подсчитываем количество:

In [7]:
df.isnull().sum()

children_number         0
days_employed        2174
age_in_years            0
education_level         0
education_id            0
family_status           0
family_status_id        0
gender                  0
employment_type         0
loan_debt_history       0
monthly_income       2174
loan_purpose            0
dtype: int64

Произведем замену пропусков в 'days_employed' на "0":

In [8]:
df['days_employed'] = df['days_employed'].fillna(0)

Замена пропущенных/нулевых значений в доходах на "0" отразится на результате исследования. Для замены пропущенных значений в категории зарплат наилучшим образом подойдет медианное значение или же можно не учитывать пропуски при ответе на гипотезу. Для нашего исследования вычислим медиану и заменим ей пропуски.

In [9]:
monthly_income_median = df['monthly_income'].median()
monthly_income_median

145017.93753253992

In [10]:
df['monthly_income'] = df['monthly_income'].fillna(145018)

Убедимся, что замены сработали:

In [11]:
df.isnull().sum()

children_number      0
days_employed        0
age_in_years         0
education_level      0
education_id         0
family_status        0
family_status_id     0
gender               0
employment_type      0
loan_debt_history    0
monthly_income       0
loan_purpose         0
dtype: int64

### Вывод

Количество пропусков в стаже и дохода одинаково. Маловероятно, что потеря этих данных связана с технической проблемой, но на всякий случай отправим запрос тем, кто нам выгружал эти данные. 

### Замена типа данных

Переводим методом .astype() в целочисленный тип значения данные столбцов трудового стажа и ежемесячного дохода для наглядного восприятия чисел:

In [12]:
df['days_employed'] = df['days_employed'].astype('int')

In [13]:
df['monthly_income'] = df['monthly_income'].astype('int')

In [14]:
df.head(10)

,children_number,days_employed,age_in_years,education_level,education_id,family_status,family_status_id,gender,employment_type,loan_debt_history,monthly_income,loan_purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Вывод

Произвели замену данных - визуальное восприятие цифр улучшилось!

### Исследуем столбцы на наличие артефактов

Посмотрим на уникальные значения столбцов и посчитаем количество наблюдений для каждого значения:

In [15]:
df['children_number'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children_number, dtype: int64

Нашли 2 странных значения: "-1" (меняем на "1") и "20" детей у аж 76 человек (вероятно из-за ошибки при вводе данных) - меняем на "2":

In [16]:
df['children_number'] = df['children_number'].replace(-1, 1)

In [17]:
df['children_number'] = df['children_number'].replace(20, 2)

In [18]:
df['children_number'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children_number, dtype: int64

Проверям следующий столбец на уникальные значения:

In [19]:
df['days_employed'].value_counts()

 0         2174
-327         16
-133         16
-438         15
-204         14
           ... 
-11220        1
-7126         1
 373727       1
 355300       1
-4058         1
Name: days_employed, Length: 9087, dtype: int64

Избавимся от отрицательных значений:

In [20]:
df['days_employed'] = df['days_employed'].abs()

Для удобства дальнейшей работы с данными добавим столбец со стажем в годах:

In [21]:
df['years_employed'] = df['days_employed'] / 365

Для наглядности переведем значения в целочисленные:

In [22]:
df['years_employed'] = df['years_employed'].astype('int')

In [23]:
df.head(10)

,children_number,days_employed,age_in_years,education_level,education_id,family_status,family_status_id,gender,employment_type,loan_debt_history,monthly_income,loan_purpose,years_employed
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,2
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,7
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,0
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,18
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,5


Видим еще одно аномальное значение - стаж 932 года. Посмотрим, сколько таких значений.
Границу определим в 85 лет как максимальный возраст выдачи кредита банком:

In [24]:
df[df['years_employed'] > 85]['years_employed'].count()

3445

Сообщаем информацию технических сотрудникам для устранения данной проблемы в будущем, так как доля ошибочных записей слишком большая. Ничего с этим не делаем, так как на наших задачах это не скажется.

Следующий столбец. Клиентов с возрастом "0" = 101 чел. Ничего с этим не делаем, так как на наших задачах это не скажется:

In [25]:
df['age_in_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: age_in_years, dtype: int64

In [26]:
df['education_level'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education_level, dtype: int64

Унифицируем все значения приведением к нижнему регистру:

In [27]:
df['education_level'] = df['education_level'].str.lower()

In [28]:
df['education_level'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education_level, dtype: int64

Количество категорий совпадает с количеством уровней образования, всё в порядке:

In [29]:
df['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [30]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Приводим значения к нижнему регистру:

In [31]:
df['family_status'] = df['family_status'].str.lower()

In [32]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Одна запись с непонятным полом - некритично:

In [33]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В следующем столбце всё в порядке:

In [34]:
df['employment_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: employment_type, dtype: int64

Здесь так же нет проблем:

In [35]:
df['loan_debt_history'].value_counts()

0    19784
1     1741
Name: loan_debt_history, dtype: int64

Проверим на всякий случай на наличие триллионов в ежемесячном доходе (всё в порядке):

In [36]:
df['monthly_income'].max()

2265604

### Обработка дубликатов

Находим дубликаты методом duplicated() и последоватльно применяем метод sum() для подсчета их количества:

In [37]:
df.duplicated().sum()

71

Удаляем дубликаты методом drop_duplicates(), последовательно используем метод reset_index() для обнуления индексов:

In [38]:
df = df.drop_duplicates().reset_index(drop=True)

In [39]:
df.duplicated().sum()

0

### Вывод

С дубликатами разобрались, не мешало бы проверить причину их появления с тех. специалистами.

### Лемматизация

Посмотрим на уникальные значения целей кредита. Понимаем, что для избавления от дубликатов, необходимо произвести лемматизацию:

In [40]:
df['loan_purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Для начала преобразуем уникальные данные в Series:

In [41]:
loan_purpose_as_series = ', '.join(df['loan_purpose'].unique())
loan_purpose_as_series

'покупка жилья, приобретение автомобиля, дополнительное образование, сыграть свадьбу, операции с жильем, образование, на проведение свадьбы, покупка жилья для семьи, покупка недвижимости, покупка коммерческой недвижимости, покупка жилой недвижимости, строительство собственной недвижимости, недвижимость, строительство недвижимости, на покупку подержанного автомобиля, на покупку своего автомобиля, операции с коммерческой недвижимостью, строительство жилой недвижимости, жилье, операции со своей недвижимостью, автомобили, заняться образованием, сделка с подержанным автомобилем, получение образования, автомобиль, свадьба, получение дополнительного образования, покупка своего жилья, операции с недвижимостью, получение высшего образования, свой автомобиль, сделка с автомобилем, профильное образование, высшее образование, покупка жилья для сдачи, на покупку автомобиля, ремонт жилью, заняться высшим образованием'

Применяем лемматизацию и счетчик:

In [42]:
lemmas = m.lemmatize(loan_purpose_as_series)
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         ', ': 37,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

### Вывод

В результате выделяются следующие цели кредита: 
- недвижимость
- жилье
- автомобиль
- образование
- свадьба

### Проведем категоризацию данных

#### Добавляем категории цели кредита

Для начала записываем названия категорий в пременную:

In [43]:
loan_purpose_category = ['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба']

Создаем функцию, возвращающую категорию цели кредита после лемматизации значения из датафрейма:

In [44]:
def categorize_loan_purpose(text):
    lemmas = m.lemmatize(text)
    for lemma in lemmas:
        
        for category in loan_purpose_category:
            if category in lemma:
                return category
            
#Проверяем работу функции
categorize_loan_purpose('берегись автомобиля')

'автомобиль'

Добавляем столбец с новыми категориями:

In [45]:
df['loan_purpose_categorized'] = df['loan_purpose'].apply(categorize_loan_purpose)

In [46]:
df.head(10)

,children_number,days_employed,age_in_years,education_level,education_id,family_status,family_status_id,gender,employment_type,loan_debt_history,monthly_income,loan_purpose,years_employed,loan_purpose_categorized
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,2,жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,7,жилье
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,0,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,18,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,5,жилье


#### Добавляем категории уровня ежемесячного дохода

С помощью метода .describe() получаем данные для разбивки на 4 категории

In [47]:
df['monthly_income'].describe()

count    2.145400e+04
mean     1.652257e+05
std      9.802100e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.450180e+05
75%      1.958132e+05
max      2.265604e+06
Name: monthly_income, dtype: float64

Создаем функцию, возвращающую название категории после обработки значения дохода:

In [48]:
def monthly_income_cat(amount):
    if amount == 145018:
        return 'импорт медианы'
    if amount <= 88946.5:
        return 'низкий'
    if amount <= 135716.0:
        return 'средний'
    if amount <= 195715.0:
        return 'высокий'
    return 'очень высокий'

#Проверяем её работу
print(monthly_income_cat(monthly_income_median))
print(monthly_income_cat(50000))
print(monthly_income_cat(100000)) 
print(monthly_income_cat(140000)) 
print(monthly_income_cat(200000)) 

высокий
низкий
средний
высокий
очень высокий


Используем функцию для категоризации данных ежемесячного дохода и добавляем столбец с новыми категориями:

In [49]:
df['monthly_income_category'] = df['monthly_income'].apply(monthly_income_cat)
df.head(10)

,children_number,days_employed,age_in_years,education_level,education_id,family_status,family_status_id,gender,employment_type,loan_debt_history,monthly_income,loan_purpose,years_employed,loan_purpose_categorized,monthly_income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,жилье,очень высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,автомобиль,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,жилье,высокий
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,образование,очень высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932,свадьба,высокий
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,2,жилье,очень высокий
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,7,жилье,очень высокий
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,0,образование,высокий
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,18,свадьба,средний
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,5,жилье,высокий


### Вывод

Добавили новые категории целей кредита и ежемесячного дохода для улучшения восприятия и дальнейшего использования при анализе.

### Шаг 3. Отвечаем на вопросы исследования <a id="step3"></a>
[*Вернуться к плану*](#step0)

#### 3.1 Есть ли зависимость между наличием детей и возвратом кредита в срок? <a id="step3.1"></a>

С помощью сводной таблицы отобразим данные по возврату кредитов в зависимости от количества детей:

In [50]:
children_number_pivot = df.pivot_table(index=['children_number'], columns='loan_debt_history', values='loan_purpose', aggfunc='count')
children_number_pivot

loan_debt_history,0,1
children_number,,
0,13028.0,1063.0
1,4410.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


Добавим столбец с отношением записей о задолженностях для ответа на вопрос в задании. Реализуем те же действия для следующих задач 3.2-3.4.

In [51]:
children_number_pivot['rate'] = children_number_pivot[1] / children_number_pivot[0]
children_number_pivot

loan_debt_history,0,1,rate
children_number,,,
0,13028.0,1063.0,0.081593
1,4410.0,445.0,0.100907
2,1926.0,202.0,0.104881
3,303.0,27.0,0.089109
4,37.0,4.0,0.108108
5,9.0,NaN,NaN


### Вывод

Зависимость есть. Наличие детей, пусть и незначительно, но негативно влияет на возврат кредита в срок. Количество детей в семье не влияет на данный показатель. Выборка по заемщикам с 5 детьми нерепрезентативна.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок? <a id="step3.2"></a>

Проводим аналогичные действия, что и в задаче 3.1. Отсортируем данные по 'rate' для наглядности:

In [52]:
family_status_pivot = df.pivot_table(index=['family_status'], columns='loan_debt_history', values='loan_purpose', aggfunc='count')
family_status_pivot['rate'] = family_status_pivot[1] / family_status_pivot[0]
family_status_pivot.sort_values('rate')

loan_debt_history,0,1,rate
family_status,,,
вдовец / вдова,896,63,0.070312
в разводе,1110,85,0.076577
женат / замужем,11408,931,0.081609
гражданский брак,3763,388,0.103109
не женат / не замужем,2536,274,0.108044


### Вывод

Зависимость есть. Риск невозврата кредита в срок выше у неженатых/незамужних людей, а также тех, кто живет в гражданском браке. 
Женатые/замужние люди, включая тех, кто развелся или потерял супруга, более ответственно обслуживают кредит.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <a id="step3.3"></a>

In [53]:
monthly_income_category_pivot = df.pivot_table(index=['monthly_income_category'], columns='loan_debt_history', values='loan_purpose', aggfunc='count')
monthly_income_category_pivot['rate'] = monthly_income_category_pivot[1] / monthly_income_category_pivot[0]
monthly_income_category_pivot.sort_values('rate')

loan_debt_history,0,1,rate
monthly_income_category,,,
очень высокий,4989,384,0.076969
низкий,2999,249,0.083028
импорт медианы,1933,170,0.087946
средний,4913,455,0.092611
высокий,4879,483,0.098996


### Вывод

Зависимость есть. Удивительно, но заемщики с низким уровнем дохода вернут кредит в срок с большей вероятностью, чем те, кто в категории с средним и, особенно, высоким уровнем дохода. Хотя разница в показателях незначительна и данное условие вряд ли должно влиять на одобрение кредита. Наименьший риск невозврата в срок - у людей с очень высоким уровнем дохода, что ожидаемо.

#### 3.4 Как разные цели кредита влияют на его возврат в срок? <a id="step3.4"></a>

In [54]:
loan_purpose_categorized_pivot = df.pivot_table(index=['loan_purpose_categorized'], columns='loan_debt_history', values='loan_purpose', aggfunc='count')
loan_purpose_categorized_pivot['rate'] = loan_purpose_categorized_pivot[1] / loan_purpose_categorized_pivot[0]
loan_purpose_categorized_pivot.sort_values('rate') #применим сортировку данных по ['rate'] для наглядности

loan_debt_history,0,1,rate
loan_purpose_categorized,,,
жилье,4152,308,0.074181
недвижимость,5877,474,0.080653
свадьба,2138,186,0.086997
образование,3643,370,0.101565
автомобиль,3903,403,0.103254


### Вывод

Выводы разделяют цели кредита на 3 категории риска:
- Низкий риск - у кредитов на покупку недвижимости, потому что это самое дорогостоящее приобретение. Принимая решение о подаче заявки на такой кредит, человек более ответственно оценивает свою способность обслуживать задолженность в долгосрочной перспективе. 
- Средний риск - у кредитов на свадьбу. Решения о получении кредитов для организации праздников/путешествий часто принимаются импульсивно.
- Высокий риск - у автокредитов и займов на образование. Автомобиль - с точки зрения сохранности вложенных денег - невыгодное приобретение, требующее впоследствии большие средства на регулярное обслуживание. Не все покупатели отдают себе в этом отчет и правильно рассчитывают свои возможности. Требования банков к заемщикам минимальны, а значит, оценка платежеспособности клиента практически не проводится. 
С кредитами на образование можно связать 2 риска: сложности с наличием постоянного заработка в параллельное с учебой время и отсутствие гарантии найти высокооплачиваемую работу после окончания обучения

### Шаг 4. Общий вывод <a id="step4"></a>
[*Вернуться к плану*](#step0)

Есть зависимость между наличием детей, семейного положения, уровнем дохода (пусть и незначительная), целью кредита и возвратом кредита в срок.

Риск по возврату кредита в срок у людей из следующих категорий:
- есть дети
- неженатые / незамужние или проживающие в гражданском браке
- кредит на автомобиль или образование

Тогда как идеальный портрет кредитора для банка - это бездетный вдовец с очень высоким уровнем дохода, приобретающий недвижимость.

#### Рекомендации

##### 1. Уделить внимание качеству записи/выгрузке данных:

- устранить причину появления пропусков, дубликатов, отрицательных значений, неподходящего типа данных, артефактов в стаже, возврасте клиентов;
- добавить классификаторы с формулой автоматического выбора категории по критериям банка и обязательные поля для заполнения в форму, которую заполняют сотрудники (ограничит категории, избавит от пропусков, дубликатов, ошибок при вводе и классификацией аналитиком); 
- перевести стаж из дней в годы для удобства восприятия и использования.

##### 2. Передать результаты анализа выше кредитным специалистам, занимающихся одобрением запросов на выдачу кредитов
